# Assignment 3

**Name:** Milagros Contreras Elizalde

**e-mail:** -- milagros.contreras8430@alumnos.udg.mx --

In [82]:
!pip install dash

In [83]:
# Import necessary libraries
import dash
from dash import dcc, html
from dash.dependencies import Input, Output
import plotly.graph_objects as go
import numpy as np
import pandas as pd
from scipy.stats import cauchy
from plotly.subplots import make_subplots

We define the Vec2d class:

In [84]:
class Vec2d:
    def __init__(self, x, y):
        self.x = x
        self.y = y

    def __add__(self, other):
        return Vec2d(self.x + other.x, self.y + other.y)

    def __mul__(self, scalar):
        return Vec2d(self.x * scalar, self.y * scalar)

# Brownian Motion

Brownian motion is the random motion of particles suspended in a medium.


In [85]:
# Functions to generate trajectories
def brownian_motion(steps, speed, start_pos):
    traj = [Vec2d(*start_pos)]
    for _ in range(steps):
        angle = np.random.uniform(0, 2 * np.pi)
        step = Vec2d(np.cos(angle), np.sin(angle)) * speed
        traj.append(traj[-1] + step)
    return traj

This function simulates Brownian motion by generating a random path for a particle. It starts from a given position, then repeatedly moves the particle in random directions for a specified number of steps.

# Random Walk

A random walk is known as a random process which describes a path including a succession of random steps in the mathematical space

In [86]:
def correlated_random_walk(steps, speed, start_pos, cauchy_coeff):
    traj = [Vec2d(*start_pos)]
    angle = 0 
    for _ in range(steps):
        angle += cauchy.rvs(scale=cauchy_coeff)  
        step = Vec2d(np.cos(angle), np.sin(angle)) * speed
        traj.append(traj[-1] + step)
    return traj

This function simulates a "correlated random walk," where the direction of movement changes gradually based on random increments.The cauchy_coeff controls how much the direction can change between steps.

# Lévy flight

 Lévy flight is a random walk in which the step-lengths have a stable distribution, a probability distribution that is heavy-tailed. When defined as a walk in a space of dimension greater than one, the steps made are in isotropic random directions. Later researchers have extended the use of the term "Lévy flight" to also include cases where the random walk takes place on a discrete grid rather than on a continuous space.

In [87]:
def levy_flight(steps, speed, start_pos, cauchy_coeff, alpha):
    traj = [Vec2d(*start_pos)]
    for _ in range(steps):
        direction = np.random.uniform(0, 2 * np.pi)
        step_length = (np.random.pareto(alpha) + 1) * cauchy_coeff 
        step = Vec2d(np.cos(direction), np.sin(direction)) * step_length * speed
        traj.append(traj[-1] + step)
    return traj


This function simulates a Lévy flight, where a particle moves in random directions with mostly short steps but sometimes makes longer, unpredictable jumps. The step lengths follow a Pareto distribution, controlled by the alpha parameter, creating a mix of frequent small movements and rare long ones.

# Functions to calculate metrics

**Path length**


This function calculates the total length of a path by summing the distances between consecutive points. It uses the Euclidean distance formula to compute these distances.

In [88]:
def calculate_path_length(trajectory):
    length = 0
    for i in range(1, len(trajectory)):
        prev, curr = trajectory[i-1], trajectory[i]
        length += np.sqrt((curr.x - prev.x)**2 + (curr.y - prev.y)**2)
    return length


**Mean squared displacement**

In statistical mechanics, the mean squared displacement (MSD, also mean square displacement, average squared displacement, or mean square fluctuation) is a measure of the deviation of the position of a particle with respect to a reference position over time. It is the most common measure of the spatial extent of random motion, and can be thought of as measuring the portion of the system "explored" by the random walker. 

In [89]:
def calculate_msd(trajectory):
    msd = []
    for tau in range(1, len(trajectory)):
        displacements = [(trajectory[i + tau].x - trajectory[i].x)**2 + (trajectory[i + tau].y - trajectory[i].y)**2 
                        for i in range(len(trajectory) - tau)]
        msd.append(np.mean(displacements))
    return msd

This function, calculate_msd, calculates the Mean Squared Displacement (MSD) for a given trajectory. MSD is a measure used in physics to quantify how much a particle's position changes over time.

**Turning Angles**

This function, calculate_turning_angles, calculates the turning angles at each point in a trajectory, which represent how much the direction of the particle changes at each step.

In [90]:
def calculate_turning_angles(trajectory):
    angles = []
    for i in range(1, len(trajectory) - 1):
        p1 = np.array([trajectory[i-1].x, trajectory[i-1].y])
        p2 = np.array([trajectory[i].x, trajectory[i].y])
        p3 = np.array([trajectory[i+1].x, trajectory[i+1].y])
        v1 = p2 - p1
        v2 = p3 - p2
        dot_prod = np.dot(v1, v2)
        cross_prod = np.cross(v1, v2)
        angle = np.arctan2(cross_prod, dot_prod)
        angles.append(angle)
    return angles

# Create the Dash application

In [91]:
app = dash.Dash(__name__)

# Define the layout of the dashboard
app.layout = html.Div([
    html.H1("Random Trajectories Dashboard", style={'text-align': 'center'}),

    # Control panel
    html.Div([
        html.Div([
            html.H3("Parameters"),
            dcc.Dropdown(
                id='trajectory-type',
                options=[
                    {'label': 'Brownian Motion', 'value': 'BM'},
                    {'label': 'Correlated Random Walk', 'value': 'CRW'},
                    {'label': 'Levy Flight', 'value': 'LF'}
                ],
                value='BM'
            ),
            html.Div([
                html.Label("Steps:"),
                dcc.Slider(
                    id='steps',
                    min=10,
                    max=1000,
                    value=500,
                    marks={i: str(i) for i in range(0, 1001, 100)},
                    step=10
                )
            ]),
            # Mueve la sección de "Initial Position" aquí
            html.Div([
                html.Label("Initial Position (x, y):"),
                dcc.Input(id='start-x', value=0, type='number'),
                dcc.Input(id='start-y', value=0, type='number')
            ]),
            # Mueve la sección de "Cauchy Coefficient" aquí
            html.Div([
                html.Label("Cauchy Coefficient:"),
                dcc.Input(id='cauchy-coeff', value=0.7, type='number')
            ]),
            html.Div([
                html.Label("Speed:"),
                dcc.Slider(
                    id='speed',
                    min=1,
                    max=10,  # Cambia el máximo a 10
                    value=1,
                    marks={i: str(i) for i in range(1, 11)},  # Actualiza los marcadores de 1 a 10
                    step=1
                )
            ]),
            html.Div([
                html.Label("Levy Exponent (α):"),
                dcc.Input(id='levy-exponent', value=1.5, type='number')
            ])
        ]),
        html.Div([
            html.H3("Metrics"),
            dcc.Dropdown(
                id='metric-type',
                options=[
                    {'label': 'Path Length', 'value': 'PL'},
                    {'label': 'Mean Squared Displacement (MSD)', 'value': 'MSD'},
                    {'label': 'Turning Angle Distribution', 'value': 'TAD'}
                ],
                value='PL'
            )
        ])
    ], style={'columnCount': 2}),

    # Visualization panel
    dcc.Graph(id='combined-plot')
])

This part of the code creates a web application using Dash to visualize random trajectories such as Brownian motion, correlated random walks, and Lévy flights.

The interface includes:

- Header: Displays the title "Random Trajectories Dashboard."
- Control Panel: Allows users to select parameters such as trajectory type, number of steps, initial position, Cauchy coefficient, speed, and Lévy exponent.
- Metrics Panel: Lets users choose the type of metric to visualize (Path Length, Mean Squared Displacement, Turning Angle Distribution).
- Visualization: A graph where the selected trajectories and metrics will be displayed.

**Callback Definition**

This is a callback function in Dash that updates the output graph ('combined-plot').
Inputs: It listens for changes in multiple input elements like:
trajectory-type (type of trajectory: Brownian Motion, Correlated Random Walk, or Levy Flight),
metric-type (metric: Path Length, MSD, or Turning Angle Distribution),
steps, speed, start-x, start-y (parameters to control trajectory behavior),
cauchy-coeff and levy-exponent (specific parameters for the Levy flight model).
When any of these inputs change, the function update_plots is called.

**Trajectory generation:** Based on the selected trajectory_type, the function generates a different type of random path:
If BM (Brownian Motion) is selected, it generates a random walk based on Brownian motion.
If CRW (Correlated Random Walk), it creates a trajectory that considers a correlation between steps using the cauchy_coeff.
If LF (Levy Flight), it uses a Lévy flight model. The levy_exponent controls the exponent for the power-law distribution.

Then the code extracts the x, y coordinates from each point in the generated trajectory and stores them in lists.
The z values represent time (or the step index), making the 3D plot display the trajectory over time.

In [94]:
# Update the trajectory and metric graphs
@app.callback(
    Output('combined-plot', 'figure'),
    [Input('trajectory-type', 'value'), Input('metric-type', 'value'), 
     Input('steps', 'value'), Input('speed', 'value'), 
     Input('start-x', 'value'), Input('start-y', 'value'), 
     Input('cauchy-coeff', 'value'), Input('levy-exponent', 'value')]
)
def update_plots(trajectory_type, metric_type, steps, speed, start_x, start_y, cauchy_coeff, levy_exponent):
    # Generate trajectories based on input parameters
    if trajectory_type == 'BM':
        trajectory = brownian_motion(steps, speed, (start_x, start_y))  # Brownian Motion
    elif trajectory_type == 'CRW':
        trajectory = correlated_random_walk(steps, speed, (start_x, start_y), cauchy_coeff)  # Correlated Random Walk
    else:
        trajectory = levy_flight(steps, speed, (start_x, start_y), cauchy_coeff, levy_exponent)  # Levy Flight
    
    # 3D trajectory plot
    x = [p.x for p in trajectory]
    y = [p.y for p in trajectory]
    z = list(range(len(trajectory)))
    
    # Create subplots in a row
    fig = make_subplots(rows=1, cols=2, subplot_titles=("3D Trajectory", "Metric"), specs=[[{"type": "scatter3d"}, {"type": "scatter"}]])

    # Add trajectory plot
    fig.add_trace(go.Scatter3d(x=x, y=y, z=z, mode='lines', name='Trajectory'), row=1, col=1)
    fig.update_yaxes(title_text="Y", row=1, col=1)
    fig.update_xaxes(title_text="X", row=1, col=1)
    fig.update_layout(scene=dict(zaxis_title='Time'))

    # Plot the selected metric
    if metric_type == 'PL':
        length = calculate_path_length(trajectory)
        fig.add_trace(go.Scatter(x=[0, len(trajectory)], y=[0, length], mode='lines', name='Length'), row=1, col=2)
        fig.update_layout(title="Path Length")
    elif metric_type == 'MSD':
        msd = calculate_msd(trajectory)
        fig.add_trace(go.Scatter(x=list(range(1, len(trajectory))), y=msd, mode='lines', name='MSD'), row=1, col=2)
        fig.update_layout(title="Mean Squared Displacement (MSD)")
    else:
        angles = calculate_turning_angles(trajectory)
        fig.add_trace(go.Histogram(x=angles, nbinsx=50, name='Turning Angles'), row=1, col=2)
        fig.update_layout(title="Turning Angle Distribution")
    
    return fig

# Run the application
if __name__ == '__main__':
    app.run_server(debug=True)


Then we create a figure with two subplots:
- Left subplot: A 3D scatter plot (for the trajectory).
- Right subplot: A 2D plot (for the selected metric).

The layout specifies that the first subplot is a 3D scatter ("scatter3d") and the second one is a 2D scatter ("scatter").

In the parte _"Add trajectory plot"_ we add the 3D plot of the trajectory. It uses go.Scatter3d to plot the path in 3D, with x, y, and z coordinates. The update_layout method adjusts axis labels (X, Y, and Time).

Then we graph the **selected metric** :
- Path Length (PL): If the user selects the path length metric, it calculates the total path length using calculate_path_length and plots it as a line in the right subplot.
- Mean Squared Displacement (MSD): If MSD is selected, it calculates the mean squared displacement over time and plots this data.
- Turning Angles (TAD): If the user selects the turning angle distribution, the angles are calculated using calculate_turning_angles, and the result is displayed as a histogram of turning angles.


Finally, we run the Dash server in debug mode, allowing live updates while the application is running

We can view it in full screen at: http://127.0.0.1:8050/


References
- Wikipedia contributors. (2024, September 27). Brownian motion. Wikipedia. https://en.wikipedia.org/wiki/Brownian_motion
- Random Walks: A Review of Algorithms and Applications. (s. f.). IEEE TRANSACTIONS ON EMERGING TOPICS IN COMPUTATIONAL INTELLIGENCE, VOL. 0, https://arxiv.org/pdf/2008.03639#:~:text=Abstract%E2%80%94A%20random%20walk%20is,as%20mathematics%20and%20computer%20science.
- Wikipedia contributors. (2024, April 25). Lévy flight. Wikipedia. https://en.wikipedia.org/wiki/L%C3%A9vy_flight
- Wikipedia contributors. (2024, September 9). Mean squared displacement. Wikipedia. https://en.wikipedia.org/wiki/Mean_squared_displacement